<a href="https://colab.research.google.com/github/sangjinsu/personalized-recommendation-system/blob/main/MF_%EA%B8%B0%EB%B0%98_%EC%B6%94%EC%B2%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 메모리 기반 알고리즘

- 메모리에 있는 데이터를 계산해서 추천하는 방식
- 개별 사용자 데이터 집중
- 원래 데이터에 충실하게 사용
- 대규모 데이터에 느리게 반응

## 모델 기반 알고리즘

- 데이터로부터 미리 모델을 구성 후 필요 시 추천하는 방식
- 전체 사용자 패턴 집중
- 대규모 데이터에 빠르게 반응
- 모델 생성 과정 오래 걸림 

## Matrix Factorization 방식의 원리 

### SGD Stochastic Gradient Decent 를 사용한  MF 알고리즘

1. 잠재 요인  K 선택
2. P, Q 행렬 초기화
3. 예측 평점 R_hat 계산
4. 실제 R 과 R_hat 간 오차 계산 및 P, Q 수정
5. 기존 오차 도달 확인 


### SGD 를 사용한 MF 기본 알고리즘

In [3]:
import os
import pandas as pd
import numpy as np

base_src = 'drive/MyDrive/recommend'
u_data_src = os.path.join(base_src, 'u.data')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(u_data_src, sep='\t',
                      names=r_cols, encoding='latin-1')

# timestamp 제거 
ratings = ratings[['user_id', 'movie_id', 'rating']].astype(int)

ratings

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1
...,...,...,...
99995,880,476,3
99996,716,204,5
99997,276,1090,1
99998,13,225,2


In [ ]:
class MF():
  def __init__(self, ratings, hyper_params):
    self.R = np.array(ratings)
    self.num_users, self.num_items = np.shape(self.R)
    self.K = hyper_params['K']
    self.alpha = hyper_params['alpha']
    self.beta = hyper_params['beta']
    self.interations = hyper_params['iterations']
    self.verbose = hyper_params['verbose']

  def rmse(self):
    xs, ys = self.R.nonzero()
    self.predictions = []
    self.errors = [] 
